In [587]:
def data_generator(info):
    a = np.sin(np.array(info)) - np.cos(np.array(info))
    a[4] = 1
    return a

def sensitivity(func, delta_variable_ini, variable_to_delta = [-1], plot = False):
    dy = np.empty(len(delta_variable_ini)) #gives problems if in y = f(x), N_y != N_x
    
    delta_variable = np.array(delta_variable_ini)
    sim_range_ini = np.linspace(0.25, 1.75, 8)
    dx = sim_range_ini[1] - sim_range_ini[0]
    sim_range = np.empty(len(delta_variable))
    
    #make sure only a certain variable is changed, instead of all variables
    delta_matrix = np.ones(len(delta_variable))
    variable_names = delta_variable_ini.index.values[variable_to_delta]
    
    if max(variable_to_delta) < 0:
        variable_to_delta = None
        variable_names = "all variables"
        print("simulating all")
    
    for i in range(len(sim_range_ini)):
        delta_matrix[variable_to_delta] *= sim_range_ini[i]
        delta_variable_temp = np.multiply(delta_variable[:,0], delta_matrix)
        sim_range = np.c_[sim_range, delta_variable_temp]
    
    #sim_range has ones as first column, this needs to be skipped
    y = func(sim_range[:,1])
    
    for i in range(len(sim_range[0, 1:])):
        new_y = func((sim_range[:,i]))
        y = np.c_[y, new_y]
        
        if i > 0:
            #dy/dx
            a = (y[:,i] - y[:, i+1])
            a = (y[:, i+1] - y[:, i]) / y[:, i]
            b = (sim_range[:, i] - sim_range[:, i+1])
            b = np.transpose(b)
            
            dy_var = (a )#/ b)
            dy = np.c_[dy, dy_var[:]]
    if(plot):
        plt.figure()
        plt.title(f'change in percentage of initial conditions [%] for changed {variable_names}')
        for p in range(len(delta_variable)):
            plt.plot(sim_range_ini[0:-1], dy[p,1:], 'o', label = p,)
        plt.xlabel(f"dx ")
        plt.legend()
        
    return sim_range_ini, dy

def fit(observed_data_x, observed_data_y, expected_data_x, expected_data_y):
    #observed is from simulation
    #expected is base truth
    
    chi_squared = 0
    
    for i in (expected_data_x):
        a = float(observed_data_y[observed_data_x == i][0])
        b = float(expected_data_y[expected_data_x == i])
        chi_squared += (a -b)**2 / b
    
    return chi_squared
    

In [589]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

info = pd.read_csv("digester_initial.csv")
info = info.transpose()
sim, dy = sensitivity(data_generator, info, [5,6])

#print(np.shape(sim), np.shape(dy))

data = pd.read_csv("testbed_values.csv")


t_rover = np.array([-1.0, 0.0, 1.0, 8.0, 11.0, 13.0])
CH4_observations = [3, 5.78, 15.5, 2.13, 5, 5] #ppbv
t = 1
observed = pd.DataFrame([t_rover + t , CH4_observations], ['x', 'y'])
data_t = data.t.round(2)

#print(observed.loc['x'])
a = fit(np.array(data_t.tolist()), np.array(data.S_gas_ch4.tolist()), observed.loc['x'], observed.loc['y'])
print(a, 'chi')

16.764461258488076 chi


In [608]:
observed = [1,2,3,5,7,5,3,1]
simulated = [1,1,1,2,3,5,7,5]
t_simulated = [0,1,2,3,4,5,6,7]

t_shifted = np.array(t_simulated) + (t_simulated[simulated.index(max(simulated))] - t_simulated[observed.index(max(observed))])

print(fit(t_simulated, simulated, t_shifted, observed))
plt.plot(t_shifted, observed, label = 'observed')
plt.plot(t_simulated, simulated, label = 'simulated')
plt.legend()

TypeError: only integer scalar arrays can be converted to a scalar index